In [3]:
from Classes.Arquivo import Arquivo
from Classes.BancoDados import BancoDados

In [ ]:
try:
    #Criação dos dataframes
    s_total = Arquivo("Arquivos CSV/suicidios-total.csv")
    homens_arma = Arquivo("Arquivos CSV/suicidios-homens-armas-fogo.csv")
    mulheres_arma = Arquivo("Arquivos CSV/suicidios-mulheres-armas-fogo.csv")
    s_homens = Arquivo("Arquivos CSV/suicidios-homens.csv")
    s_mulheres = Arquivo("Arquivos CSV/suicidios-mulheres.csv")
    s_regiao = Arquivo("Arquivos CSV/suicidios-região.csv")

    #Inspeção
    if s_total.verifica and homens_arma.verifica and mulheres_arma.verifica and s_homens.verifica and s_mulheres.verifica and s_regiao.verifica:
        print("Arquivos CSV inspecionados com sucesso!")
    else:
        print(f"""
Status de verificação de cada arquivo:
suicidios-total.csv = {s_total.verifica}
suicidios-homens-armas-fogo.csv = {homens_arma.verifica}
suicidios-mulheres-armas-fogo.csv = {mulheres_arma.verifica}
suicidios-homens.csv = {s_homens.verifica}
suicidios-mulheres.csv = {s_mulheres.verifica}
suicidios-região.csv = {s_regiao.verifica}
        """)
except Exception as e:
    print(f"Erro ao inspecionar o arquivo: {e}")

Arquivos CSV inspecionados com sucesso!
